In [21]:
from astropy.io import fits
from astropy.table import Table
from spectral_cube import SpectralCube
import matplotlib.pyplot as plt
import pandas as pd
import open3d as o3d

In [15]:
path = ".data/reduced_TAN_C14.fits"

In [16]:
hdul = fits.open(path)
table = Table(hdul[0].data)


In [33]:
output = []

for col in table.columns:
    df = Table(table[col]).to_pandas()
    df.columns = range(len(df.columns))
    df_stacked = df.stack().reset_index()
    df_stacked.columns = ['y', 'x', 'value']
    df_stacked['z'] = int(col.split('col')[1])
    output.append(df_stacked)

In [38]:
df = pd.concat(output)[['x', 'y', 'z', 'value']].reset_index(drop=True)

In [40]:
df.describe()

,x,y,z,value
count,8.572500e+06,8.572500e+06,8.572500e+06,8.572500e+06
mean,7.450000e+01,2.210000e+02,7.450000e+01,3.512622e-01
std,4.330031e+01,1.099849e+02,4.330031e+01,2.328835e+00
min,0.000000e+00,3.100000e+01,0.000000e+00,-4.922788e-01
25%,3.700000e+01,1.260000e+02,3.700000e+01,-2.067636e-02
50%,7.450000e+01,2.210000e+02,7.450000e+01,9.568406e-03
75%,1.120000e+02,3.160000e+02,1.120000e+02,4.640185e-02
max,1.490000e+02,4.110000e+02,1.490000e+02,1.265473e+02


In [41]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(df.iloc[:,:3].values.tolist())
volume = df['value'].values
volume_normalized = (volume - volume.min()) / (volume.max() - volume.min())
colors = plt.cm.plasma(volume)[:, :3]  # Use a colormap (e.g., viridis)
pcd.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([pcd])